# California housing prices 
.
Versão com o PySpark do kernel [California housing prices](https://github.com/robson-rsp/datascience/blob/main/regression/california_housing_prices.ipynb). Este notebook foi escrito no ambiente do Databricks e exportado como arquivo IPython .ipynb. Os textos daqui são cópias dos originais do link.
---
A tarefa neste kernel é criar um modelo de Machine Learning que estime preços de imóveis no estado da Califórnia baseado em características, como:

*   latitude/longitude: Localização do imóvel
*   housing_median_age: Idade média dos imóveis naquela quadra
*   totalRooms: Número total de cômodos dentro de um quarteirão
*   totalBedrooms: Número total de quartos dentro de um quarteirão
*   population: Número total de pessoas que moram em uma quadra
*   households: Número total de habitações em uma quadra
*   medianIncome: Renda média das famílias de uma quadra (medida em dezenas de milhares de dólares americanos)
*   medianHouseValue: Valor médio dos imóveis de uma quadra (medido em dólares americanos)
*   oceanProximity: Localização do imóvel em relação ao oceano/mar

Obs. A média populacional desses quarteirões é de 600 até 3000 pessoas.

Fonte: [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [0]:
from pyspark.sql.functions import *

In [0]:

housing = spark.read.format("csv") \
          .option("inferSchema", "True") \
          .option("header", "True") \
          .option("sep", ",") \
          .load("/FileStore/tables/housing.csv")

# EDA

In [0]:
housing.sample(fraction=0.01).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.26,37.85,52.0,2491.0,474.0,1098.0,468.0,3.0750,213500.0,NEAR BAY
1,-122.28,37.81,17.0,1237.0,462.0,762.0,439.0,0.9241,177500.0,NEAR BAY
2,-122.20,37.78,45.0,1766.0,332.0,869.0,327.0,4.5893,163500.0,NEAR BAY
3,-122.18,37.78,50.0,1642.0,322.0,713.0,284.0,3.2984,160700.0,NEAR BAY
4,-122.18,37.76,52.0,754.0,175.0,447.0,165.0,3.9063,93800.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
207,-118.96,34.19,16.0,1807.0,346.0,587.0,296.0,1.9811,162500.0,<1H OCEAN
208,-118.85,34.14,24.0,1999.0,244.0,759.0,247.0,8.7657,366300.0,<1H OCEAN
209,-118.71,34.27,26.0,990.0,223.0,719.0,232.0,3.1630,179400.0,<1H OCEAN
210,-121.77,38.68,43.0,2559.0,598.0,1820.0,591.0,2.1927,107900.0,INLAND


In [0]:
print(f'shape={housing.count(), len(housing.columns)}')

shape=(20640, 10)


In [0]:
housing.summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Verificando a ocorrência de campos NaN.

In [0]:
for name in housing.columns:
    mask = housing[name].isNull()
    print(f'{name}: {housing.where(mask).count()}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


# Data visualization

Os gráficos foram criados utilizando as ferramentas de visualização nativas do Databricks. O resultado não pode ser visto em um arquivo .ipynb

# Train/Test split

In [0]:
train_set, test_set = housing.randomSplit([0.8, 0.2])

# Feature engineering

In [0]:
from pyspark.ml.feature import Imputer, StandardScaler, StringIndexer, VectorAssembler

## Categorical features

**Etapa ##:** Atributos nominais: Codificar

In [0]:
si = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_')
si_model = si.fit(train_set)
train_set = si_model.transform(train_set)
test_set  = si_model.transform(test_set)

## Numerical features

**Etapa ##:** NaN values: Imputar

In [0]:
im = Imputer(strategy='mean', inputCol='total_bedrooms', outputCol='total_bedrooms_')
im_model = im.fit(train_set)
train_set = im_model.transform(train_set)
test_set  = im_model.transform(test_set)

**Etapa ##:** Os atributos 'total_rooms' e 'total_bedrooms' não fazem sentido se eu deixá-los da forma como estão. 'total_rooms' refere-se à quantidade total de cômodos que há em cada quadra. 'total_bedrooms' refere-se à quantidade total de quartos que há em cada quadra. Vou criar novos atributos a partir dos existentes. Vou fazer isso dentro de uma classe para usar a classe Pipeline.
1. **mean_bedrooms_households:** Quantidade média de quartos por habitação.
1. **mean_rooms_households:** Quantidade média de cômodos por habitação.
1. **mean_population_households:** Quantidade média de moradores por imóvel.

In [0]:
train_set = train_set.withColumn('mean_bedrooms_households', train_set['total_bedrooms_'] / train_set['households'])
train_set = train_set.withColumn('mean_rooms_households', train_set['total_rooms'] / train_set['households'])
train_set = train_set.withColumn('mean_population_households', train_set['population'] / train_set['households'])

test_set = test_set.withColumn('mean_bedrooms_households', test_set['total_bedrooms_'] / test_set['households'])
test_set = test_set.withColumn('mean_rooms_households', test_set['total_rooms'] / test_set['households'])
test_set = test_set.withColumn('mean_population_households', test_set['population'] / test_set['households'])

**Etapa ##:** Unir todas as colunas em uma única que será utilizada pelos modelos de Machine Learning do PySpark. Essa etapa dever ser feita antes da padronização dos atributos com StandardScaler.

In [0]:
cols = ['longitude', 'latitude', 'housing_median_age', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity_', 'total_bedrooms_', 'mean_bedrooms_households', 'mean_rooms_households', 'mean_population_households']

va = VectorAssembler(inputCols=cols, outputCol='X')
train_set = va.transform(train_set)
test_set  = va.transform(test_set)

**Etapa ##:** Padronização dos atributos com StandardScaler.

In [0]:
ss = StandardScaler(inputCol='X', outputCol='X_')
ss_model = ss.fit(train_set)
train_set = ss_model.transform(train_set)
test_set  = ss_model.transform(test_set)

**Etapa ##:** Selecionar X e y dos conjuntos de treino e teste. Vou renomear "X_" para "X"  e "median_house_value" para "y" assim os conjuntos ficarão de acordo com o padrão de nomenclatura.

In [0]:
train_set = train_set.select('X_', 'median_house_value')
test_set  = test_set.select('X_', 'median_house_value')

train_set = train_set.withColumnRenamed(existing='X_', new='X')
test_set  = test_set.withColumnRenamed(existing='X_', new='X')

train_set = train_set.withColumnRenamed(existing='median_house_value', new='y')
test_set  = test_set.withColumnRenamed(existing='median_house_value', new='y')

# Model training


In [0]:
from pyspark.ml.regression import GBTRegressor, LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

## Baseline model
Como o PySpark não possui uma classe como DummyRegressor do Scikit Learn, vou reproduzir o resultado que ela teria caso existisse. O padrão dela é prever sempre a média aritmética da label do conjunto de treinamento. Portanto, é isso que farei, mas sem o uso de classes.

In [0]:
m = train_set.select(mean('y')).collect()[0][0] # média aritmética de y
test_set = test_set.withColumn('y_mean', lit(m))
test_set.show(5)

+--------------------+-------+-----------------+
|                   X|      y|           y_mean|
+--------------------+-------+-----------------+
|[-61.963930099210...|66900.0|206888.4430120482|
|[-61.963930099210...|68400.0|206888.4430120482|
|[-61.953952813951...|70000.0|206888.4430120482|
|[-61.948964171322...|64600.0|206888.4430120482|
|[-61.943975528693...|82800.0|206888.4430120482|
+--------------------+-------+-----------------+
only showing top 5 rows



In [0]:
re = RegressionEvaluator(labelCol='y', predictionCol='y_mean', metricName='mse')
print(f'mse: {re.evaluate(test_set)}')

mse: 13548796044.305721


## LinearRegression

In [0]:
lr = LinearRegression(featuresCol='X', labelCol='y', elasticNetParam=0.0)
lr_model = lr.fit(train_set)
y_pred = lr_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show(5)

mse: 9.329412801621729e-15
+--------------------+-------+-----------------+-----------------+
|                   X|      y|           y_mean|       prediction|
+--------------------+-------+-----------------+-----------------+
|[-61.963930099210...|66900.0|206888.4430120482|66899.99999997109|
|[-61.963930099210...|68400.0|206888.4430120482|68399.99999997586|
|[-61.953952813951...|70000.0|206888.4430120482|69999.99999988341|
|[-61.948964171322...|64600.0|206888.4430120482|64599.99999989171|
|[-61.943975528693...|82800.0|206888.4430120482|82799.99999989271|
+--------------------+-------+-----------------+-----------------+
only showing top 5 rows



## RandomForestRegressor

In [0]:
rf = RandomForestRegressor(featuresCol='X', labelCol='y')
rf_model = rf.fit(train_set)
y_pred = rf_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show(5)

mse: 557296371.2876711
+--------------------+-------+-----------------+-----------------+
|                   X|      y|           y_mean|       prediction|
+--------------------+-------+-----------------+-----------------+
|[-61.963930099210...|66900.0|206888.4430120482|83571.59371068899|
|[-61.963930099210...|68400.0|206888.4430120482|88729.25904116343|
|[-61.953952813951...|70000.0|206888.4430120482|83645.48616994816|
|[-61.948964171322...|64600.0|206888.4430120482|81187.49753192508|
|[-61.943975528693...|82800.0|206888.4430120482|96589.74454958757|
+--------------------+-------+-----------------+-----------------+
only showing top 5 rows



## GBTRegressor

In [0]:
gb = GBTRegressor(featuresCol='X', labelCol='y')
gb_model = gb.fit(train_set)
y_pred = gb_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show(5)

mse: 29209444.338794507
+--------------------+-------+-----------------+-----------------+
|                   X|      y|           y_mean|       prediction|
+--------------------+-------+-----------------+-----------------+
|[-61.963930099210...|66900.0|206888.4430120482| 69961.4257910274|
|[-61.963930099210...|68400.0|206888.4430120482| 69961.4257910274|
|[-61.953952813951...|70000.0|206888.4430120482| 69961.4257910274|
|[-61.948964171322...|64600.0|206888.4430120482|58385.36457629232|
|[-61.943975528693...|82800.0|206888.4430120482|80973.35616469014|
+--------------------+-------+-----------------+-----------------+
only showing top 5 rows



## Best model
O melhos modelo, de longe, é o LinearRegression.